# Library Tools and Packages


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# General Analysis


In [2]:
data = pd.read_csv('binance_data.csv',header=None)
columns = ['timestamp','open','high','low','close','volume']
data.columns = columns
data["timestamp"] = pd.to_datetime(data["timestamp"])
data=data.drop(['timestamp'],axis = 1 )
data

,open,high,low,close,volume
0,97930.13,97982.54,97928.20,97982.53,18.27246
1,97982.54,97999.98,97944.00,97949.99,9.10211
2,97949.99,97984.25,97949.99,97984.24,10.24976
3,97984.24,97984.25,97942.00,97942.00,9.39547
4,97942.01,97967.99,97892.06,97892.06,21.62757
...,...,...,...,...,...
414,94800.00,94872.02,94768.00,94808.02,36.25977
415,94808.02,94866.89,94736.00,94816.98,56.18272
416,94816.98,94914.61,94800.00,94914.61,50.36162
417,94914.61,94971.83,94824.00,94863.49,42.76734


In [3]:
data.isna().sum()

open      0
high      0
low       0
close     0
volume    0
dtype: int64

In [4]:
data.duplicated().sum()

0

# Data scaling & Splitting 

In [5]:
X_raw = data[['open', 'high', 'low', 'volume']].values
y_raw = data[['close']].values

In [6]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler() 

X = scaler_X.fit_transform(X_raw)
y = scaler_y.fit_transform(y_raw)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Neural Network - ANN

In [8]:
input_neurons = X_train.shape[1]  # Number of predictors which is 4 because I got 4 predictors in the data set 
hidden_neurons = 7  # Neurons in the hidden layer -> Randomly select 7; we can change it.
output_neurons = 1  # Single output (price) -> As the output, we need the closing prices in unsee time units, which implies one node is enough. 
learning_rate = 0.01
epochs = 1000

In [9]:
np.random.seed(10)
W1 = np.random.randn(input_neurons, hidden_neurons) * 0.01
b1 = np.zeros((1, hidden_neurons))
W2 = np.random.randn(hidden_neurons, output_neurons) * 0.01
b2 = np.zeros((1, output_neurons))

# Since the goal is to begin the neural network in a neutral state, no numerical values for bias were used. 

In [10]:
def relu(z):
    return np.maximum(0, z)
def relu_derivative(z):
    return np.where(z > 0, 1, 0)

#  1) Use relu activation for the hidden layer and just linear activation for the output layer since the desired output is price, 
# which is a numerical value; if sigmoid was used, it would yield a probability value. 

In [11]:
# Detailed in a separate PDF 
# Training loop
for epoch in range(epochs):
    # Forward propagation 
    Z1 = np.dot(X_train, W1) + b1 
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2  # Linear activation for regression
    A2 = Z2

    # Compute loss (Mean Squared Error)
    loss = np.mean((A2 - y_train) ** 2)

    # Backward propagation 
    dA2 = 2 * (A2 - y_train) / y_train.shape[0]  # Derivative of MSE w.r.t A2
    dZ2 = dA2  # Linear activation derivative
    dW2 = np.dot(A1.T, dZ2)
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X_train.T, dZ1)
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    # Update weights and biases
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

    # Print loss every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

print("Training complete.")

Epoch 0, Loss: 0.2681
Epoch 100, Loss: 0.0801
Epoch 200, Loss: 0.0767
Epoch 300, Loss: 0.0765
Epoch 400, Loss: 0.0762
Epoch 500, Loss: 0.0757
Epoch 600, Loss: 0.0749
Epoch 700, Loss: 0.0735
Epoch 800, Loss: 0.0713
Epoch 900, Loss: 0.0678
Training complete.


# Prediction Part

In [12]:
def predict(X_new):
    Z1 = np.dot(X_new, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    return Z2  # Linear activation gives raw predictions

In [13]:
scaled_predictions = predict(X_test)
predictions = scaler_y.inverse_transform(scaled_predictions)  # Inverse transform to get original prices

In [14]:
y_test_original = scaler_y.inverse_transform(y_test)

In [15]:
test_loss = np.mean((predictions - y_test_original) ** 2)
print(f"Test Loss (MSE): {test_loss:.4f}")

Test Loss (MSE): 719858.3745


In [16]:
for i in range(5):  # Display 5 predictions
    print(f"Actual: {y_test_original[i][0]:.2f}, Predicted: {predictions[i][0]:.2f}")

Actual: 94987.99, Predicted: 95941.03
Actual: 95887.99, Predicted: 96024.88
Actual: 97687.98, Predicted: 96206.27
Actual: 97023.51, Predicted: 96146.15
Actual: 96139.99, Predicted: 96044.45
